In [2]:
### this is for running in local ###
import os
try:
    os.environ['HTTP_PROXY']='http://185.46.212.90:80'
    os.environ['HTTPS_PROXY']='http://185.46.212.90:80'
    print ("proxy_exported")
except:
    None

proxy_exported


In [3]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

In [4]:
from datasets import load_dataset
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

In [5]:
dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

/usr/local/lib/python3.8/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [6]:
model_name = "microsoft/phi-2"
bnb_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.float16,)
model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,trust_remote_code=True)
model.config.use_cache = False

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["Wqkv","out_proj","fc1","fc2"]
    )

In [9]:
from transformers import TrainingArguments

new_model = "phi-2-fine-tuned"
output_dir = "./results"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=500,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=500,
    warmup_ratio=0.05,
    group_by_length=True,
    lr_scheduler_type="constant",
    gradient_checkpointing=False,
)

In [10]:
from trl import SFTTrainer

max_seq_length = 300

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
for name, module in trainer.model.named_modules():
    if "ln" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [ ]:
# Run text generation pipeline with our next model
prompt = "Where is Africa ?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

- Copying the Qlora model to gdrive for future use

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cp -r '/content/phi-2-sharded-bf16-custom' "/content/gdrive/MyDrive/ERA1/s27_qlora/llama7b_finetuning"

## Inference from copied model. We will merge the qlora model with original one & save it to gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cp -r "/content/gdrive/MyDrive/ERA1/s27_qlora/llama7b_finetuning/phi-2-sharded-bf16-custom" "/content"

In [ ]:
model_path = "/content/phi-2-sharded-bf16-custom-merged"  # change to your preferred path
model_name = "microsoft/phi-2"
# device_map = {"": 1}

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
    # device_map=device_map,
)

In [ ]:
from peft import PeftModel
new_model = "phi-2-sharded-bf16-custom"
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
!cp -r '/content/phi-2-sharded-bf16-custom-merged' "/content/gdrive/MyDrive/ERA1/s27_qlora/llama7b_finetuning"

## Inference from copied model. We will then download this merged model & perform inference

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cp -r "/content/gdrive/MyDrive/ERA1/s27_qlora/llama7b_finetuning/phi-2-sharded-bf16-custom-merged" "/content"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/phi-2-sharded-bf16-custom-merged"

model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
from transformers import pipeline

prompt = "What was the role of indian revolutionaries in indian independence movement ?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=500)
result = gen(prompt)
print(result[0]['generated_text'])

In [ ]:
prompt = "What is a black hole ?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=200)
result = gen(prompt)
print(result[0]['generated_text'])

In [ ]:
result